In [1]:
import cv2
import numpy as np
import os
from keras.models import load_model
import pickle
#import matplotlib.pyplot as plt

In [2]:
face_cascade_Path = "haarcascade_frontalface_default.xml"
model=load_model("mask_detector.model")
faceCascade = cv2.CascadeClassifier(face_cascade_Path)
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer.yml')
font = cv2.FONT_HERSHEY_SIMPLEX
id = 0
# names related to ids: The names associated to the ids: 1 for Mohamed, 2 for Jack, etc...
names = pickle.load(open('name.pkl', 'rb'))
results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

In [7]:
cam = cv2.VideoCapture(0)
cam.set(3, 640)
cam.set(4, 480)
# Min Height and Width for the  window size to be recognized as a face
minW = 0.1 * cam.get(3)
minH = 0.1 * cam.get(4)
img  = 0
gray = 0

In [8]:
while True:
    
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        img,
        minNeighbors=5,
        minSize=(int(minW), int(minH)),
    )

    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        id, confidence = recognizer.predict(gray[y:y + h, x:x + w])
        
        face_img = img[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(224,224))
        reshaped=resized.reshape(1,224,224,3)/255
        final = np.vstack([reshaped])
        result=model.predict(final)
        
        label=np.argmax(result,axis=1)[0]
        if (confidence < 100):
            face_name = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            # Unknown Face
            id = "Who are you ?"
            confidence = "  {0}%".format(round(100 - confidence))
        if label == 0:
            cv2.putText(img, "mask detected", (x + 5, y - 5), font, 1, (255, 255, 255), 2)
        else:
            cv2.putText(img, str(face_name), (x + 5, y - 5), font, 1, (255, 255, 255), 2)
            

        #cv2.putText(img, "yes", (x + 5, y - 5), font, 1, (255, 255, 255), 2)
        #cv2.putText(img, str(confidence), (x + 5, y + h - 5), font, 1, (255, 255, 0), 1)
    cv2.imshow('camera', img)
    # Escape to exit the webcam / program
    k = cv2.waitKey(10) & 0xff
    if k == 27:
        break
print("\n [INFO] Exiting Program.")
cam.release()
cv2.destroyAllWindows()


 [INFO] Exiting Program.
